
# Data curation

Man Ho Wong | m.wong@pitt.edu | Feb 27th, 2022

This notebook search for the datasets needed for this project in the following database:
- [CHILDES](https://childes.talkbank.org/)  
  *Reference:* MacWhinney, B. (2000). The CHILDES Project: Tools for analyzing talk. Third Edition. Mahwah, NJ: Lawrence Erlbaum Associates.

I may not need datasets from [Wordbank](http://wordbank.stanford.edu/) as I found that CHILDES probably has all the data I need.

I will also explore the datasets on the way to get a sense of the contents and the structures of the datasets (such as participant information, annotations, data format, etc.), as well as some basic statistics about the datasets. After that, I will identify the information I need in the datasets and compile the data for data processing later.

---

In [1]:
from tqdm import tqdm
import pickle

# data = pickle.load(open('data/childes/corpus_info.pkl', 'rb'))
# search_result = data[0]
# data_idx = data[1]


# 1 Searching for suitable corpora in CHILDES

CHILDES is a multilingual database containing corpora with transcriptions, audio recordings and/or video recordings of child speech and child-directed speech (CDS) at different developmental stages. Each corpus has a separate directory for each participant, and each directory contains the recording transcripts stored in CHAT formats. ([Example](https://childes.talkbank.org/access/Eng-NA/Brown.html))

For this project, I will need to collect the transcipts for both the child speech and the associated CDS. Additionally, I will need the participant information (i.e. child age, sex and socioeconomic status (SES), mother's education) and some basic annotations of the words (i.e. morphemes and lexical categories). Participant information can be found in the header of each CHAT file as the metadata of the file. Annotation information can be found as dependent tiers embedded in the transcription.

Let's take a quick look at a sample CHAT file first to see how the data is organized.


## 1.1 Acccessing data in CHAT file

### Reading CHAT file

The `PyLangAcq` package allows users to read CHAT files directly from a zip file. You can download and install it with the following code:  
`$ pip install --upgrade pylangacq`

For documentation, you can visit their [website](https://pylangacq.org/).

I will use the Brown Corpus of CHILDES as an example below. The corpus has been downloaded from [here](https://childes.talkbank.org/data/Eng-NA/Brown.zip) and stored under `data_samples/childes/Brown.zip`. There are three folders in this corpus, each folder contains a dataset (a collection of CHAT files) for each child:

```
Brown.zip/  
    |--Adam/  
    |--Eve/  
    |--Sarah/
```

I will use the `read_chat()` function of `PyLangAcq` to read all the CHAT files in the dataset `Adam`:

In [2]:
import pylangacq

# Read CHAT files in the dataset 'Adam' in 'Brown.zip':
path = 'data_samples/childes/Brown.zip'
adam = pylangacq.read_chat(path, 'Adam')

print(type(adam))
print('Number of CHAT files:', adam.n_files())

<class 'pylangacq.chat.Reader'>
Number of CHAT files: 55


In [3]:
# Ages when recordings were made
print('Ages (year, month, day):', adam.ages())  # output: a list of tuples

Ages (year, month, day): [(2, 3, 4), (2, 3, 18), (2, 4, 3), (2, 4, 15), (2, 4, 30), (2, 5, 12), (2, 6, 3), (2, 6, 17), (2, 7, 1), (2, 7, 14), (2, 8, 1), (2, 8, 16), (2, 9, 4), (2, 9, 18), (2, 10, 2), (2, 10, 16), (2, 10, 30), (2, 11, 13), (2, 11, 28), (3, 0, 11), (3, 0, 25), (3, 1, 9), (3, 1, 26), (3, 2, 9), (3, 2, 21), (3, 3, 4), (3, 3, 18), (3, 4, 1), (3, 4, 18), (3, 5, 1), (3, 5, 15), (3, 5, 29), (3, 6, 9), (3, 7, 7), (3, 8, 1), (3, 8, 14), (3, 8, 26), (3, 9, 16), (3, 10, 15), (3, 11, 1), (3, 11, 14), (4, 0, 14), (4, 1, 15), (4, 2, 17), (4, 3, 9), (4, 4, 1), (4, 4, 13), (4, 5, 11), (4, 6, 24), (4, 7, 1), (4, 7, 29), (4, 9, 2), (4, 10, 2), (4, 10, 23), (5, 2, 12)]


As shown above, `read_chat()` read the CHAT files and creates a `Reader` object. This is a `dataclass` storing data and metadata across all the CHAT files in `Adam`. You can access the data stored in the `Reader` by calling the appropriate methods/ attributes, such as `.n_files()` for number of CHAT files in the dataset. For example, `Adam` has 55 CHAT files. We can also get the ages when recordings were made by calling `.ages()`. Let's see what other information we can get from the `Reader` object in the next section.

### Accessing metadata stored in a CHAT file

Metadata such as age range, date of recording, participants, etc. are stored in the header of each CHAT file. We can access such information by retrieving the `.header()` attribute. Here is the header for the first CHAT file in `adam`:

In [4]:
adam.headers()[0]

{'UTF8': '',
 'PID': '11312/c-00015632-1',
 'Languages': ['eng'],
 'Participants': {'CHI': {'name': 'Adam',
   'language': 'eng',
   'corpus': 'Brown',
   'age': '2;03.04',
   'sex': 'male',
   'group': 'TD',
   'ses': 'MC',
   'role': 'Target_Child',
   'education': '',
   'custom': ''},
  'MOT': {'name': 'Mother',
   'language': 'eng',
   'corpus': 'Brown',
   'age': '',
   'sex': 'female',
   'group': '',
   'ses': '',
   'role': 'Mother',
   'education': '',
   'custom': ''},
  'URS': {'name': 'Ursula_Bellugi',
   'language': 'eng',
   'corpus': 'Brown',
   'age': '',
   'sex': '',
   'group': '',
   'ses': '',
   'role': 'Investigator',
   'education': '',
   'custom': ''},
  'RIC': {'name': 'Richard_Cromer',
   'language': 'eng',
   'corpus': 'Brown',
   'age': '',
   'sex': '',
   'group': '',
   'ses': '',
   'role': 'Investigator',
   'education': '',
   'custom': ''},
  'COL': {'name': 'Colin_Fraser',
   'language': 'eng',
   'corpus': 'Brown',
   'age': '',
   'sex': '',
   

The output above, is a multilevel `dictionary`. To retrieve a specific piece of information we need, we can use the `dictionary` keys as usual.  
Let's check if 'Adam' is a male as its biblical name suggests:

In [5]:
adam.headers()[0]['Participants']['CHI']['sex']

'male'


### Accessing annotations

Next, I will check what kinds of annotation information are stored in each CHAT file. I will use the `.tokens()` method to access the tokens with annotation information. This method creates a `list` of `Token` objects:

In [6]:
tokens = adam.tokens()
tokens[:5]  # first five tokens

[Token(word='play', pos='n', mor='play', gra=Gra(dep=1, head=2, rel='MOD')),
 Token(word='checkers', pos='n', mor='checker-PL', gra=Gra(dep=2, head=0, rel='INCROOT')),
 Token(word='.', pos='.', mor='', gra=Gra(dep=3, head=2, rel='PUNCT')),
 Token(word='big', pos='adj', mor='big', gra=Gra(dep=1, head=2, rel='MOD')),
 Token(word='drum', pos='n', mor='drum', gra=Gra(dep=2, head=0, rel='INCROOT'))]

Each `Token` is a `dataclass` with attributes (e.g. `word`,`pos`, etc.) as shown in the above example.  
Annotations for each word are stored as the `Token`'s attributes (i.e. attributes other than `word`):

In [7]:
print("Second token in 'Adam':")
print('Word: {}\nMorpheme: {}\nPart of speech: {}'.format(
    tokens[1].word, tokens[1].mor, tokens[1].pos))

Second token in 'Adam':
Word: checkers
Morpheme: checker-PL
Part of speech: n



## 1.2 Search strategy

Now that we know what kinds of information are stored in each CHAT file and how we can access them, we can start planning our search accordingly.

### Identifying the scope of search

There are dozens of English corpora in CHILDES. We don't need to download them all at once just to look for the corpora we need. `PyLangAcq` allows user to read a corpus directly with the corpus's URL. We can read the corpora one by one, and keep only the ones we need. To get the URLs for all the North American (NA) English corpora, one can use a web scraping tool to get all the links from the database's website and look for the corpus URLs from there. However, a much faster way is to take advantage of the TalkBank's [browsable database](https://sla.talkbank.org/TBB/childes):  
1. navigate to CHILDES's North American English datasets (Eng-NA)
2. copy the list of corpora directly to a spreadsheet program and save it as a `csv` file (example: `data_samples/childes/eng_NA_corpus_list.csv`)
3. construct the URL for each corpus simply from the name of the corpus we get from step 2 (see below)

CHILDES has a very well organized structure. Each corpus has the same URL format as follow:  
`https://childes.talkbank.org/data/LANGUAGE/NAME_OF_CORPUS.zip`  
For example, the URL for the Brown Corpus is: https://childes.talkbank.org/data/Eng-NA/Brown.zip

Let's see what corpora will be included in our search:

In [8]:
import pandas as pd

# Read the list of corpora into a Pandas Series:
corpus_list = pd.read_csv('data/childes/eng_NA_corpus_list.csv', 
                          header=None, index_col=False, squeeze = True)

print('There are {} NA English corpora in CHILDES.'.format(len(corpus_list)))
print('Here are the first 10 corpora:')
corpus_list.head(10)

There are 47 NA English corpora in CHILDES.
Here are the first 10 corpora:


0        Bates
1    Bernstein
2        Bliss
3        Bloom
4     Bohannon
5    Braunwald
6        Brent
7        Brown
8        Clark
9    Demetras1
Name: 0, dtype: object

There are 47 NA English corpora in CHILDES that we can potentially use! Since there is a huge volume of data that we have to evaluate, we need a good strategy to make the data search more efficient. For this project, I will develop a search stretegy similar to a data collection process following PRISMA (Preferred Reporting Items for Systematic Reviews and Meta-Analyses) statement ([flow diagram](https://view.officeapps.live.com/op/view.aspx?src=http%3A%2F%2Fwww.prisma-statement.org%2Fdocuments%2FPRISMA_2020_flow_diagram_new_SRs_v1.docx&wdOrigin=BROWSELINK), [publication](https://journals.plos.org/plosmedicine/article?id=10.1371/journal.pmed.1003583)), which is commonly used in medicine and life sciences for meta-analyses.

### Searching data in three phases

Instead of evaluating each of the CHAT files from all 47 corpora directly, I will perform the search in three phases, where each phase narrows down the scope of search. As the scope of search is becoming more manageable, I will use search criteria that are more specific. 

1. **Identification**  
    Identify relevant corpora fitting a set of basic criteria  
2. **Screening**  
    Screen for CHAT files containing the information we need  
3. **Refining**  
    Refine the dataset by filtering CHAT files with more specific criteria

## 1.3 Run the search

### Phase 1: Identification

I will first search for the corpora meeting the basic requirements of the project, i.e. corpora involving relevant participants (child for child speech analysis and mother for CDS analysis).

Here are the search criteria:
- Participants: data includes child or mother
- Child information: data contains child age, sex and socioeconomic status (SES) information
- Mother information: data contains socioeconomic status (SES), education information

To access participant information, we can first read the data into a `Reader` object, and then retrieve the header information from the `.headers` attribute.

#### About efficiency

Reading all corpora at once into one `Reader` object is not memory-efficient, but reading each single CHAT file into a `Reader` involves a large number of iterations. Therefore, I will read and evaluate one corpus at a time. Once a CHAT file in the corpus fits the criteria, the rest of the CHAT files will not be evaluated and I will move on to the next corpus (because CHAT files belonging to the same corpus are supposed to come from the same study contain similar information).

The `.headers` of a `Reader` contains a `list` of headers of *all* the CHAT files in the dataset.  
Let's look at our favorite dataset 'Adam' again. Here are the headers of the first two CHAT files in 'Adam':

In [9]:
adam.headers()[:2]

[{'UTF8': '',
  'PID': '11312/c-00015632-1',
  'Languages': ['eng'],
  'Participants': {'CHI': {'name': 'Adam',
    'language': 'eng',
    'corpus': 'Brown',
    'age': '2;03.04',
    'sex': 'male',
    'group': 'TD',
    'ses': 'MC',
    'role': 'Target_Child',
    'education': '',
    'custom': ''},
   'MOT': {'name': 'Mother',
    'language': 'eng',
    'corpus': 'Brown',
    'age': '',
    'sex': 'female',
    'group': '',
    'ses': '',
    'role': 'Mother',
    'education': '',
    'custom': ''},
   'URS': {'name': 'Ursula_Bellugi',
    'language': 'eng',
    'corpus': 'Brown',
    'age': '',
    'sex': '',
    'group': '',
    'ses': '',
    'role': 'Investigator',
    'education': '',
    'custom': ''},
   'RIC': {'name': 'Richard_Cromer',
    'language': 'eng',
    'corpus': 'Brown',
    'age': '',
    'sex': '',
    'group': '',
    'ses': '',
    'role': 'Investigator',
    'education': '',
    'custom': ''},
   'COL': {'name': 'Colin_Fraser',
    'language': 'eng',
    'cor

Each header is a multi-level `dictionary`. To access the participant information in each of the headers, one can loop through each header and index into the sub-level dictionary associated with the key `Participants`. For example:

In [16]:
for h in adam.headers():
    if 'MOT' in h ['Participants']:  # search criterion
        print('At least one file involves mother as a participant in the dataset "Adam"!')
        break  # Break the loop once the search criterion is met

At least one file involves mother as a participant in the dataset "Adam"!


Let's begin the first phase of the search:

In [15]:
search_result = []  # To store a list of corpora matching the criteria

# Search each corpus in the list:
for corpus_name in tqdm(corpus_list):  # tqdm for progress bar
    
    # Download URL for the corpus:
    corpus_url = 'https://childes.talkbank.org/data/Eng-NA/'+corpus_name+'.zip'
    
    # read the corpus into a Reader object:
    corpus = pylangacq.read_chat(corpus_url)
   
    # Search criteria:  
    # - Child ('CHI') or mother ('MOT') is included as participant.
    # - Info about Child's SES or mother's SES/education is provided.
    # Note: Check if 'CHI' or 'MOT' is present before checking SES/education info
    #       (if the condition before 'and' is False, the second condition won't
    #       be evaluated. This is not only more efficient, but also prevents
    #       error when evaluating the second condition if 'CHI'/'MOT' is absent).
    
    for h in corpus.headers():
        if (
            (('CHI' in h['Participants']) and (h['Participants']['CHI']['ses'] != '')) or 
            (('MOT' in h['Participants']) and (h['Participants']['MOT']['ses'] != '')) or 
            (('MOT' in h['Participants']) and (h['Participants']['MOT']['education'] != ''))
        ):
            search_result.append(corpus_name)  # store corpus name in 'search_result'
            break  # Break the for loop and move on to the next corpus

print('Search completed! {} corpora found!'.format(len(search_result)))

100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [06:27<00:00,  8.24s/it]

Search completed! 13 corpora found!


Let's see which corpora contain the data we need:

In [17]:
print('\n{} corpora matching the criteria:'.format(len(search_result)))
search_result


13 corpora matching the criteria:


['Bates',
 'Bernstein',
 'Brown',
 'Clark',
 'Demetras2',
 'Gleason',
 'HSLLD',
 'Hall',
 'Hicks',
 'Nelson',
 'NewmanRatner',
 'Post',
 'VanHouten']

Nice! We have narrowed down the number of corpora we need to process from 47 to 13.  
Next, I will download the zip files for the corpora to my local drive and extract them to the folder `data/childes/`.

In [ ]:
import os
import urllib.request
import zipfile

# Where zip files will be stored locally
target_dir = 'data/childes/corpus_zip/'

# Create a folder to store the zip files
os.mkdir(target_dir)

for corpus_name in search_result:
    
    print('Downloading and extracting {}...'.format(corpus_name))
    
    # Corpus's download URL:
    corpus_url = 'https://childes.talkbank.org/data/Eng-NA/'+corpus_name+'.zip'
    
    # Path in local drive:
    zip_path = target_dir + corpus_name + '.zip'
    
    # Download corpus from URL
    urllib.request.urlretrieve(corpus_url, zip_path)
    
    # Extract zip file
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall('data/childes')
        
print('Done! Zip files were stored in {} \nand extracted to {}.'
      .format(target_dir, 'data/childes'))

### Phase 2: Screening

We have downloaded all the corpora containing any number of CHAT files matching our search criteria. Since not all the files in the same corpus match the search criteria, I will exclude those files which do not match the criteria from my analysis. I will create a `DataFrame` as an index pointing to the files that meet the search criteria:

In [ ]:
file_path_list = []
corpus_name_list = []
chi_name_list = []
chi_age_d_list = []
chi_age_m_list = []
chi_sex_list = []
chi_group_list = []
chi_ses_list = []
mot_edu_list = []

for corpus_name in tqdm(search_result):  # tqdm for progress bar
    corpus = pylangacq.Reader.from_dir('data/childes/'+corpus_name)
    
    for f in corpus:
        
        if (
            # Participants must include both the child and the mother.
            (('CHI' in f.headers()[0]['Participants']) and 
            ('MOT' in f.headers()[0]['Participants'])) 
            
            and
            
            # Dataset should include child's or mother's SES/education.
            (( 'ses' in f.headers()[0]['Participants']['MOT']) or
             (f.headers()[0]['Participants']['CHI']['ses'] != '') or
             (f.headers()[0]['Participants']['MOT']['education'] != ''))
        ):
            
            age_raw     = f.ages()[0]  # age in raw format
            chi_age_d   = (age_raw[0]*12 + age_raw[1])*30 + age_raw[2]
            chi_age_m   = age_raw[0]*12 + age_raw[1] + round(age_raw[2]/30,1)

            # Include only children <= 72 months:
            if chi_age_m <= 72: 
            
                file_path   = f.file_paths()[0].replace('\\','/')
                file_path   = file_path[0].lower() + file_path[1:]
                
                corpus_name = f.headers()[0]['Participants']['CHI']['corpus']
                chi_name    = f.headers()[0]['Participants']['CHI']['name']
                chi_sex     = f.headers()[0]['Participants']['CHI']['sex']
                chi_group   = f.headers()[0]['Participants']['CHI']['group']
                chi_ses     = f.headers()[0]['Participants']['CHI']['ses']
                mot_edu     = f.headers()[0]['Participants']['MOT']['education']

                file_path_list.append(file_path) 
                corpus_name_list.append(corpus_name)
                chi_name_list.append(chi_name)
                chi_age_d_list.append(chi_age_d)
                chi_age_m_list.append(chi_age_m)
                chi_sex_list.append(chi_sex)
                chi_group_list.append(chi_group)
                chi_ses_list.append(chi_ses)
                mot_edu_list.append(mot_edu)

data_idx = pd.DataFrame({'file_path':file_path_list,
                         'corpus':corpus_name_list,
                         'name':chi_name_list,
                         'age_d':chi_age_d_list,
                         'age_m':chi_age_m_list,
                         'sex':chi_sex_list,
                         'group':chi_group_list,
                         'ses':chi_ses_list,
                         'mot_edu':mot_edu_list,
                        })

print('Done!')
data_idx

### Phase 3: Refining

This will be done on the fly during data analysis.

---

# 2 Basic statistics

## 2.1 Token count

We can use the methods `.tokens()` and `.utterances` to access the token and utterance information stored in the `Reader` objects. Let's see how many tokens are three in these corpora:

In [ ]:
all_tokens = []  # To store a list of lists of tokens for all corpora
token_sum = 0    # Total token counter

for corpus in search_result:
    files = data_idx[data_idx.corpus==corpus].file_path
    reader = pylangacq.Reader.from_files(files)
    # Get token info and store them in 'all_tokens':
    tokens = reader.tokens()  # list of Token objects
    all_tokens.append(tokens)
    
    # Print result
    print('Token count in {}: {}'.format(corpus, len(tokens)))
    token_sum = token_sum + len(tokens)

# Print result
print('\nTotal token count: {}'.format(token_sum))


## 2.2 Utterance count

Similarly, we can access the utterance infomation stored in each `Reader` object. For example:

In [ ]:
file = data_idx.file_path[0]  # first file in 'data_idx'
reader = pylangacq.Reader.from_files([file])
reader.utterances()[3]  # 4th utterance

The example above shows the second utterance and their annotation information in the first corpus ('Bates'), including the words, the speaker and more. To look at child-directed speech (CDS) specifically, we can set the `.utterances()`'s `participants` option to `MOT`:

In [ ]:
reader.utterances(participants='MOT')[0] # first utterance by mother

Next, I will look at how many utterances are there in the data. How many utterances are there in the child speech, and how many in the mother's CDS?

In [ ]:
all_utt_chi = []  # To store a list of lists of child utterances for all corpora
all_utt_mot = []  # To store a list of lists of mother utterances for all corpora
utt_chi_sum = 0   # Total child utterance counter
utt_mot_sum = 0   # Total mother utterance counter
utt_sum = 0       # Total utterance counter

for corpus in search_result:
    files = data_idx[data_idx.corpus==corpus].file_path
    reader = pylangacq.Reader.from_files(files)
    # Get utterances and store them in 'all_utt_chi' or 'all_utt_mot':
    utt_chi = reader.utterances(participants='CHI')  # list of Utterance object
    utt_mot = reader.utterances(participants='MOT')  # list of Utterance object
    all_utt_chi.append(utt_chi)
    all_utt_mot.append(utt_mot)
    
    # Print results
    print('Child utterance count in {}: {}'.format(corpus, len(utt_chi)))
    print('Mother utterance count in {}: {}'.format(corpus, len(utt_mot)))
    utt_chi_sum = utt_chi_sum + len(utt_chi)
    utt_mot_sum = utt_mot_sum + len(utt_mot)

utt_sum = utt_sum + utt_chi_sum + utt_mot_sum
utt_chi_pc = round (utt_chi_sum/utt_sum*100, 2)  # Child utterance percentage
utt_mot_pc = round (utt_mot_sum/utt_sum*100, 2)  # Mother utterance percentage

# Print results
print('\n-------------------------------------------------\n')
print('\nTotal child utterance count: {}'.format(utt_chi_sum))
print('\nTotal mother utterance count: {}'.format(utt_mot_sum))
print('\nTotal utterance count: {}'.format(utt_sum))
print('\nUtterance count percentage: {}% by child; {}% by mother'
      .format(utt_chi_pc, utt_mot_pc))

## 2.3 Demographics of participants

Vocabulary development can be influenced by a participant's background, such as age, sex, SES, mother's education and developmental groups (e.g. typically developing children). Let's look at these factors first to get a better idea of how the demographics looks like and what kinds of information are available in the data for further analysis.

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot') # ggplot style


ses_count = data_idx.groupby('ses').aggregate('count').file_path[1:]

plt.pie(ses_count, labels=ses_count.index, autopct='%1.1f%%')
plt.show()

In [ ]:
data_idx.groupby('ses').aggregate('count').file_path[1:].i

---

# 3 Data objects for further analysis

The above code have created several data objects ready for further analysis. I will pickle these objects so that I don't need create these objects again every time.

In [ ]:
import pickle

data = [search_result, data_idx2]

f = open('data/childes/corpus_info.pkl', 'wb')  
pickle.dump(data, f, -1)
f.close()